In [14]:
import sys
sys.path.append('/dfs/scratch1/simran/tutorial/contextual-embeddings/')
import os
import random
import ujson
import argparse
import pandas as pd
import pickle
import json
import csv
from collections import defaultdict, OrderedDict
import jsonlines
pd.set_option('display.max_colwidth', -1)


In [15]:
def load_mentions(file): 
    lines = []
    with jsonlines.open(file) as f: 
        for line in f: 
            new_line = {
                'id': line['id'],
                'sentence': line['sentence'],
                'aliases': line['aliases'], 
                'spans': line['spans'],
                'qids': line['qids'],
                'anchor': line['anchor'],
                'sent_idx_unq': line['sent_idx_unq']
            }
            lines.append(new_line)
    return pd.DataFrame(lines)

In [16]:
def get_span(span):
    span = span.split(":")
    span_s = int(span[0])
    span_e = int(span[1])
    return span_s, span_e

In [21]:
base_dir = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_cidr_model/bootleg_09132020/'
expt_dir = 'subjobj_candjen_only_2/'
file_fullsent = 'all_sentence_w_bootmentions.jsonl'
file_subjobj = 'candgen_prepped_for_bootinput.jsonl'

In [22]:
full_sent_filename = '{}{}{}'.format(base_dir, expt_dir, file_fullsent) #TO FILL IN 
candgen_filename = '{}{}{}'.format(base_dir, expt_dir, file_subjobj) #TO FILL IN 

sentence_df = load_mentions(full_sent_filename)
candgen_df = load_mentions(candgen_filename)

candgen_df.rename(columns={'aliases':'aliases_candgen'}, inplace=True)
candgen_df.rename(columns={'spans':'spans_candgen'}, inplace=True)
candgen_df.rename(columns={'qids':'qids_candgen'}, inplace=True)
candgen_df.rename(columns={'anchor':'anchors_candgen'}, inplace=True)

aliases_candgen = {}
spans_candgen = {}
qids_candgen = {}
anchors_candgen = {}
for ind, row in candgen_df.iterrows():
    aliases_candgen[row['id']] = row['aliases_candgen']
    spans_candgen[row['id']] = row['spans_candgen']
    qids_candgen[row['id']] = row['qids_candgen']
    anchors_candgen[row['id']] = row['anchors_candgen']
    
sentence_df['aliases_candgen'] = sentence_df['id'].map(aliases_candgen)
sentence_df['spans_candgen'] = sentence_df['id'].map(spans_candgen)
sentence_df['qids_candgen'] = sentence_df['id'].map(qids_candgen)
sentence_df['anchors_candgen'] = sentence_df['id'].map(anchors_candgen)

In [23]:
print(sentence_df.columns)

for ind, row in sentence_df.iterrows():
    print(row)
    if ind > 5:
        break

Index(['id', 'sentence', 'aliases', 'spans', 'qids', 'anchor', 'sent_idx_unq',
       'aliases_candgen', 'spans_candgen', 'qids_candgen', 'anchors_candgen'],
      dtype='object')
id                 61b3a5c8c9a882dcfcd2                                                                                                                                                                                                                        
sentence           Tom Thabane resigned in October last year to form the All Basotho Convention ( ABC ) , crossing the floor with 17 members of parliament , causing constitutional monarch King Letsie III to dissolve parliament and call the snap election .
aliases            [tom thabane, all basotho convention, abc, crossing the floor, members of parliament, constitutional monarch, king letsie iii, dissolve parliament, snap election]                                                                          
spans              [0:2, 10:13, 14:15, 17:20, 22:25,

# create hybrid data

In [24]:
# print(base_mentions_df['sentence'])

In [25]:
pd.set_option('display.max_colwidth', -1)
import math 

span_dict_overall = {}
qid_dict_overall = {}
alias_dict_overall = {}
anchor_dict_overall = {}

for ind, row in sentence_df.iterrows():
    hybrid_spans = {}
    
    candgen_dict = {}
    if type(row['spans_candgen']) != float:
        for i in range(len(row['spans_candgen'])):
            span, qid, alias, anchor = row['spans_candgen'][i], row['qids_candgen'][i], row['aliases_candgen'][i], row['anchors_candgen'][i]
            ss, se = get_span(span)
            candgen_dict[i] = {"qid": qid, "ss": ss, "se":se, "alias": alias, 'anchor': anchor}
    
    for i in range(len(row['spans'])):
        span, qid, alias, anchor = row['spans'][i], row['qids'][i], row['aliases'][i], row['anchor'][i]
        ss, se = get_span(span)
        
        to_skip = 0
        for ind, cg_val in candgen_dict.items():
            if (cg_val['ss'] >= ss and cg_val['ss'] <= se) or (cg_val['se'] >= ss and cg_val['se'] <= se): # cand span start intersects
                # add from cand gen
                span_to_add = '{}:{}'.format(cg_val['ss'], cg_val['se'])
                hybrid_spans[span_to_add] = {"qid": cg_val['qid'], "alias": cg_val['alias'], 'anchor': cg_val['anchor']}
                to_skip = 1
                
        if not to_skip:       
            # add from full sentence
            span_to_add = '{}:{}'.format(ss, se)
            hybrid_spans[span_to_add] = {"qid": qid, "alias": alias, 'anchor': anchor}
                
    # collect for this row            
    final_span = []
    final_qid = []
    final_alias = []
    final_anchor = []
    for k, v in hybrid_spans.items():
        final_span.append(k)
        final_qid.append(v['qid'])
        final_alias.append(v['alias'])
        final_anchor.append(v['anchor'])
        
    # collect for all the data
    span_dict_overall[row['id']] = final_span
    qid_dict_overall[row['id']] = final_qid
    alias_dict_overall[row['id']] = final_alias
    anchor_dict_overall[row['id']] = final_anchor
          




In [26]:
sentence_df['spans'] = sentence_df['id'].map(span_dict_overall)
sentence_df['qids'] = sentence_df['id'].map(qid_dict_overall)
sentence_df['aliases'] = sentence_df['id'].map(alias_dict_overall)
sentence_df['anchor'] = sentence_df['id'].map(anchor_dict_overall)

In [27]:
sentence_df.drop(columns=['spans_candgen','qids_candgen', 'aliases_candgen', 'anchors_candgen'])
                          

,id,sentence,aliases,spans,qids,anchor,sent_idx_unq
0,61b3a5c8c9a882dcfcd2,"Tom Thabane resigned in October last year to form the All Basotho Convention ( ABC ) , crossing the floor with 17 members of parliament , causing constitutional monarch King Letsie III to dissolve parliament and call the snap election .","[tom thabane_0, all basotho convention_0, abc, crossing the floor, members of parliament, constitutional monarch, king letsie iii, dissolve parliament, snap election]","[0:2, 10:13, 14:15, 17:20, 22:25, 27:29, 29:32, 33:35, 38:40]","[Q983971, Q1346132, Q169889, Q5188683, Q11010, Q41614, Q57537, Q741182, Q25052149]","[True, True, True, True, True, True, True, True, True]",0
1,61b3a65fb9b7111c4ca4,"In 1983 , a year after the rally , Forsberg received the so-called `` genius award '' from the John D. and Catherine T. MacArthur Foundation .","[forsberg_1, genius award, john d_1]","[9:10, 14:16, 19:21]","[Q348416, Q1543268, Q29552]","[True, True, True]",1
2,61b3a65fb9aeb61c81e7,This was among a batch of paperback Oxford World 's Classics that I was given as a reward for reading and commenting on a manuscript for OUP .,"[batch, oxford world_2, reward, reading, oup_2]","[4:5, 7:9, 17:18, 19:20, 26:27]","[Q2960829, Q362, Q4372150, Q18729, Q217595]","[True, True, True, True, True]",2
3,61b3a65fb9c9956eccbc,"The latest investigation was authorized after the Supreme Court in 2007 found DCC and its founder , Jim Flavin , guilty of selling DCC 's ( EURO ) 106 million ( then $ 130 million ) stake in Fyffes after Flavin -- also a Fyffes director at the time -- received inside information about bad Fyffes news in the pipeline .","[supreme court, dcc, jim, flavin, dcc, euro, stake, fyffes, flavin, fyffes, inside information, fyffes_3, pipeline]","[7:9, 12:13, 17:18, 18:19, 23:24, 26:27, 36:37, 38:39, 40:41, 44:45, 51:53, 55:56, 59:60]","[Q11201, Q5261092, Q5928, Q504395, Q5261092, Q46, Q4356599, Q3091339, Q504395, Q3091339, Q854623, Q3091339, Q725864]","[True, True, True, True, True, True, True, True, True, True, True, True, True]",3
4,61b3a65fb9197aba87ff,"The event is a response to a White House immigration reform proposal in March , said Chung-Wha Hong , executive director of the New York Immigration Coalition .","[white house, immigration reform, new york immigration coalition_4, immigration]","[7:9, 9:11, 23:27, 25:26]","[Q11696, Q5156979, Q155654, Q597260]","[True, True, True, True]",4
...,...,...,...,...,...,...,...
106259,e779865fb9651c3ba307,"`` If you look around Europe at the moment everybody 's got their steel helmets on as the scaffolding 's crumbling down , and we 've seen no indications that there 's a banking crisis in Switzerland , '' said James Nason , spokesman for the Swiss Bankers Association .","[look around, europe, steel helmets, scaffolding, banking crisis, switzerland, james nason_90750, swiss bankers association_90750]","[3:5, 5:6, 13:15, 18:19, 33:35, 36:37, 40:42, 46:49]","[Q2527642, Q46, Q767924, Q233320, Q806663, Q39, Q23685, Q798804]","[True, True, True, True, True, True, True, True]",106259
106260,e779889f60ce6ec6f3e2,"In contrast , Morrisons slumped to the bottom of the index as it dropped 494 percent in value to 2809 pence Bolland , 50 , credited with reviving the fortunes of British supermarket chain Morrisons since his appointment there as chief executive in 2006 , will replace Briton Stuart Rose at the start of 2010 , M&S said in a statement .","[morrisons_90751, index, bolland_90751, the fortunes, british, morrisons, chief executive, briton, stuart rose, ms]","[3:4, 10:11, 21:22, 28:30, 31:32, 34:35, 40:42, 47:48, 48:50, 56:57]","[Q922344, Q22765, Q911912, Q353075, Q30, Q922344, Q484876, Q145, Q2618302, Q1494]","[True, True, True, True, True, True, True, True, True, True]",106260
106261,e77985f2e816f8df8386,"I also know other authors , including D.S. Greenberg , who was once president of the AAAS , and even the names also mentioned in the article , plus many articles in the AAAAS journal , also c

In [28]:
outfile = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_cidr_model/bootleg_09132020/subjobj_candjen_only_2/hybrid_with_bootmentions'
sentence_df.to_json(r'{}.json'.format(outfile),orient='records')
with open('{}.json'.format(outfile)) as infile:
    data = json.load(infile)
    print(len(data))
    
all_out = '{}.jsonl'.format(outfile)
with open(all_out, 'w') as outfile:
    for d in data:
        json.save(d, outfile)
        outfile.write('\n')

106264


# now combine alias2qid.json

In [29]:
base_map_dir = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_cidr_model/bootleg_downloads/entity_db/entity_mappings/'
sentence_map_name = 'alias2qids_wiki.json'
candgen_map_name = 'alias2qid_candgen2.json'


In [30]:
sentence_map_file = "{}{}".format(base_map_dir, sentence_map_name)
with open(sentence_map_file) as sent_map:
    sent_content = json.load(sent_map)
    print(len(sent_content))
    
cand_file = "{}{}".format(base_map_dir, candgen_map_name)
with open(cand_file) as cand_map:
    cand_content = json.load(cand_map)
    print(len(cand_content))


7970529
204632


In [31]:
# print(type(sent_content))
# for k, v in sent_content.items():
#     print(k, v)
#     break
    
merged_dict = {key: value for (key, value) in sent_content.items()}
for key, value in cand_content.items():
    merged_dict[key] = value
print(len(merged_dict))

8175161


In [32]:
outfile = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_cidr_model/bootleg_downloads/entity_db/entity_mappings/alias2qid_hybrid2.json'
with open(outfile, 'w') as out:
    json.save(merged_dict, out)
